In [8]:
import sys
import os
import json
import numpy as np
import torch
import matplotlib.pyplot as plt
from pathlib import Path
from mmengine.config import Config
from opentad.datasets import build_dataset, build_dataloader
from opentad.datasets.base import SlidingWindowDataset, PaddingDataset, filter_same_annotation
from opentad.datasets.pku import PkuSlidingDataset, PkuPaddingDataset

sys.path.append(str(Path.cwd()))
print("lib import completed")

lib import completed


In [ ]:
# 설정 파일 경로
config_path = "configs/adatad/pku_mmd/e2e_pku_mmd_videomae_s_768x1_160_adapter.py"

# 설정 파일 로드
cfg = Config.fromfile(config_path)
print(f"설정 파일 로드 완료: {config_path}")
print(f"설정 키: {list(cfg.keys())}")

# 데이터셋 설정 확인
if hasattr(cfg, "dataset"):
    dataset_cfg = cfg.dataset
    print(f"데이터셋 설정 키: {list(dataset_cfg.keys())}")
    
    # 훈련 데이터셋 설정 확인
    if hasattr(dataset_cfg, 'train'):
        train_cfg = dataset_cfg.train
        print(f"훈련 데이터셋 설정:")
        print(f"  타입: {train_cfg.get('type', 'Unknown')}")
        print(f"  어노테이션 파일: {train_cfg.get('ann_file', 'Unknown')}")
        print(f"  데이터 경로: {train_cfg.get('data_path', 'Unknown')}")
        print(f"  파이프라인 개수: {len(train_cfg.get('pipeline', []))}")
else:
    print("데이터셋 설정을 찾을 수 없습니다.")

설정 파일 로드 완료: configs/adatad/pku_mmd/e2e_pku_mmd_videomae_s_768x1_160_adapter copy.py
설정 키: ['annotation_train', 'annotation_val', 'annotation_test', 'class_map', 'video_dir', 'block_list', 'window_size', 'dataset', 'evaluation', 'model', 'scale_factor', 'chunk_num', 'solver', 'optimizer', 'scheduler', 'inference', 'post_processing', 'workflow', 'work_dir']
데이터셋 설정 키: ['train', 'val', 'test']
훈련 데이터셋 설정:
  타입: PkuPaddingDataset
  어노테이션 파일: data/PKU-MMD/pku_train.json
  데이터 경로: F:/dataset/pku-mmd/rgb
  파이프라인 개수: 13


In [10]:
# 훈련 데이터셋 생성
print("훈련 데이터셋 생성 중...")
train_dataset = build_dataset(cfg.dataset.train)
print(f"훈련 데이터셋 생성 완료: {len(train_dataset)} 개 샘플")

# 훈련 데이터로더 생성 (train.py와 동일한 방식)
train_loader = build_dataloader(
    train_dataset,
    rank=0,
    world_size=1,
    batch_size=2,
    num_workers=0,
    shuffle=True,
    drop_last=False
)
print("훈련 데이터로더 생성 완료")

훈련 데이터셋 생성 중...
training subset: 831 videos
훈련 데이터셋 생성 완료: 831 개 샘플
훈련 데이터로더 생성 완료


In [11]:
# 검증 데이터셋 생성
print("검증 데이터셋 생성 중...")
val_dataset = build_dataset(cfg.dataset.val)
print(f"검증 데이터셋 생성 완료: {len(val_dataset)} 개 샘플")

# 검증 데이터로더 생성
val_loader = build_dataloader(
    val_dataset,
    rank=0,
    world_size=1,
    batch_size=2,
    num_workers=0,
    shuffle=False,
    drop_last=False
)
print("검증 데이터로더 생성 완료")

검증 데이터셋 생성 중...
validation subset: 111 videos, truncated as 1958 windows.
검증 데이터셋 생성 완료: 1958 개 샘플
검증 데이터로더 생성 완료


In [12]:
# 테스트 데이터셋 생성
print("테스트 데이터셋 생성 중...")
test_dataset = build_dataset(cfg.dataset.test)
print(f"테스트 데이터셋 생성 완료: {len(test_dataset)} 개 샘플")

# 테스트 데이터로더 생성
test_loader = build_dataloader(
    test_dataset,
    rank=0,
    world_size=1,
    batch_size=2,
    num_workers=0,
    shuffle=False,
    drop_last=False
)
print("테스트 데이터로더 생성 완료")

테스트 데이터셋 생성 중...
testing subset: 132 videos, truncated as 2370 windows.
테스트 데이터셋 생성 완료: 2370 개 샘플
테스트 데이터로더 생성 완료


In [14]:
print("훈련 데이터로더 테스트...")
try:
    for batch_idx, batch in enumerate(train_loader):
        if batch_idx > 0:      # 첫 배치만 확인
            break

        print("첫 번째 배치 로드 성공")
        print("배치 키 :", list(batch.keys()))

        # ───────── inputs ─────────
        if "inputs" in batch:
            inp = batch["inputs"]
            print(f"inputs shape : {inp.shape}  dtype={inp.dtype}")
            print(f"inputs range : {inp.min().item():.3f} ~ {inp.max().item():.3f}")

        # ───────── masks ──────────
        if "masks" in batch:
            m = batch["masks"]
            print(f"masks shape  : {m.shape}  dtype={m.dtype}")
            print(f"masks ratio  : {m.float().mean().item():.3f}")  # 1 의 비율

        # ───────── GT segments ────
        if "gt_segments" in batch:
            segs = batch["gt_segments"]
            seg0 = segs[0] if isinstance(segs, list) else segs[0]
            print(f"gt_segments[0] shape : {seg0.shape}")
            print(f"gt_segments[0] value : {seg0[:3]}")

        # ───────── GT labels ──────
        if "gt_labels" in batch:
            lbls = batch["gt_labels"]
            lbl0 = lbls[0] if isinstance(lbls, list) else lbls[0]
            print(f"gt_labels[0] shape   : {lbl0.shape}")
            print(f"gt_labels[0] sample  : {lbl0[:10]}")
            print(f"label id min~max     : {lbl0.min().item()} ~ {lbl0.max().item()}")

except Exception as e:
    print("훈련 데이터로더 오류:", e)
    import traceback; traceback.print_exc()


훈련 데이터로더 테스트...


첫 번째 배치 로드 성공
배치 키 : ['inputs', 'masks', 'gt_segments', 'gt_labels', 'metas']
inputs shape : torch.Size([2, 1, 3, 512, 160, 160])  dtype=torch.uint8
inputs range : 0.000 ~ 255.000
masks shape  : torch.Size([2, 512])  dtype=torch.bool
masks ratio  : 1.000
gt_segments[0] shape : torch.Size([4, 2])
gt_segments[0] value : tensor([[  0.,  39.],
        [123., 272.],
        [274., 352.]])
gt_labels[0] shape   : torch.Size([4])
gt_labels[0] sample  : tensor([28, 43, 16,  4], dtype=torch.int32)
label id min~max     : 4 ~ 43


In [16]:
print("검증 데이터로더 테스트...")
try:
    for batch_idx, batch in enumerate(val_loader):
        if batch_idx > 0:      # 첫 배치만 확인
            break

        print("첫 번째 배치 로드 성공")
        print("배치 키 :", list(batch.keys()))

        # ───────── inputs ─────────
        if "inputs" in batch:
            inp = batch["inputs"]
            print(f"inputs shape : {inp.shape}  dtype={inp.dtype}")
            print(f"inputs range : {inp.min().item():.3f} ~ {inp.max().item():.3f}")

        # ───────── masks ──────────
        if "masks" in batch:
            m = batch["masks"]
            print(f"masks shape  : {m.shape}  dtype={m.dtype}")
            print(f"masks ratio  : {m.float().mean().item():.3f}")  # 1 의 비율

        # ───────── GT segments ────
        if "gt_segments" in batch:
            segs = batch["gt_segments"]
            seg0 = segs[0] if isinstance(segs, list) else segs[0]
            print(f"gt_segments[0] shape : {seg0.shape}")
            print(f"gt_segments[0] value : {seg0[:3]}")

        # ───────── GT labels ──────
        if "gt_labels" in batch:
            lbls = batch["gt_labels"]
            lbl0 = lbls[0] if isinstance(lbls, list) else lbls[0]
            print(f"gt_labels[0] shape   : {lbl0.shape}")
            print(f"gt_labels[0] sample  : {lbl0[:10]}")
            print(f"label id min~max     : {lbl0.min().item()} ~ {lbl0.max().item()}")

except Exception as e:
    print("검증 데이터로더 오류:", e)
    import traceback; traceback.print_exc()


검증증 데이터로더 테스트...
첫 번째 배치 로드 성공
배치 키 : ['inputs', 'masks', 'gt_segments', 'gt_labels', 'metas']
inputs shape : torch.Size([2, 1, 3, 512, 160, 160])  dtype=torch.uint8
inputs range : 0.000 ~ 255.000
masks shape  : torch.Size([2, 512])  dtype=torch.bool
masks ratio  : 1.000
gt_segments[0] shape : torch.Size([3, 2])
gt_segments[0] value : tensor([[229., 293.],
        [377., 423.],
        [462., 488.]])
gt_labels[0] shape   : torch.Size([3])
gt_labels[0] sample  : tensor([25, 17, 26], dtype=torch.int32)
label id min~max     : 17 ~ 26


In [27]:
print("\n훈련 데이터셋 첫 번째 샘플 상세 분석...")
first_sample = train_dataset[0]
print("샘플 키 :", list(first_sample.keys()))

# ───────── 메타데이터 ─────────
meta = first_sample["metas"]                     # 메타 정보는 여기!
print("\n[메타데이터]")
for k, v in meta.items():
    print(f"  {k}: {v}")

# 편의 변수
window_start = meta.get("windows_start_frame",   # 슬라이딩 윈도우용
                        meta.get("window_start_frame", 0))
snippet_stride = meta.get("snippet_stride", 1)
offset_frames  = meta.get("offset_frames", 0)

# ───────── 비디오 정보 ─────────
print(f"\n비디오명        : {meta.get('video_name', 'N/A')}")
print(f"총 프레임 수     : {meta.get('duration', 'N/A')}")
print(f"FPS             : {meta.get('fps', 'N/A')}")
print(f"스니펫 스트라이드: {snippet_stride}")
print(f"윈도우 시작 프레임: {window_start}")

# ───────── GT 세그먼트 & 라벨 ─────────
if {"gt_segments", "gt_labels"} <= first_sample.keys():
    segs = first_sample["gt_segments"]
    labs = first_sample["gt_labels"]
    print(f"\nGT 세그먼트 개수: {len(segs)}  |  GT 라벨 개수: {len(labs)}")

    for i, (seg, lab) in enumerate(zip(segs, labs)):
        print(f"  ▸ 세그먼트 {i:02d} (윈도우 좌표) : {seg.tolist()}")
        full_seg = seg * snippet_stride + window_start + offset_frames
        print(f"    → 전체 프레임 좌표          : {full_seg.tolist()}  |  라벨 {lab}")
else:
    print("GT 정보가 없습니다.")



훈련 데이터셋 첫 번째 샘플 상세 분석...
샘플 키 : ['inputs', 'masks', 'gt_segments', 'gt_labels', 'metas']

[메타데이터]
  video_name: 0002-L
  data_path: F:/dataset/pku-mmd/rgb
  fps: 30.0
  duration: 4462
  snippet_stride: 1
  window_start_frame: 0
  window_size: 512
  offset_frames: 0

비디오명        : 0002-L
총 프레임 수     : 4462
FPS             : 30.0
스니펫 스트라이드: 1
윈도우 시작 프레임: 0

GT 세그먼트 개수: 2  |  GT 라벨 개수: 2
  ▸ 세그먼트 00 (윈도우 좌표) : [26.0, 45.0]
    → 전체 프레임 좌표          : [26.0, 45.0]  |  라벨 8
  ▸ 세그먼트 01 (윈도우 좌표) : [326.0, 377.0]
    → 전체 프레임 좌표          : [326.0, 377.0]  |  라벨 21


In [18]:
# 클래스맵 파일 로드
print("\n클래스맵 및 라벨 매칭 확인...")
class_map_path = "data/PKU-MMD/class_map.txt"
try:
    with open(class_map_path, 'r') as f:
        class_names = [line.strip() for line in f.readlines()]
    print(f"클래스 개수: {len(class_names)}")
    print(f"첫 번째 클래스: {class_names[0]} (인덱스 0)")
    print(f"마지막 클래스: {class_names[-1]} (인덱스 {len(class_names)-1})")
    
    # 원본 어노테이션 파일 확인
    ann_file = "data/PKU-MMD/pku_train.json"
    with open(ann_file, 'r') as f:
        original_annotations = json.load(f)
    
    # 첫 번째 비디오의 어노테이션 확인
    if len(original_annotations) > 0:
        first_video = original_annotations[0]
        print(f"\n첫 번째 비디오: {first_video['video_name']}")
        print(f"어노테이션 개수: {len(first_video['annotations'])}")
        
        for i, anno in enumerate(first_video['annotations'][:3]):  # 처음 3개만
            label_name = anno['label']
            segment = anno['segment']
            print(f"  어노테이션 {i}: {label_name} - {segment}")
            
            # 클래스맵에서 인덱스 찾기
            if label_name in class_names:
                class_idx = class_names.index(label_name)
                print(f"    클래스맵 인덱스: {class_idx}")
            else:
                print(f"    클래스맵에 없음!")
                
except Exception as e:
    print(f"클래스맵 로드 오류: {e}")


클래스맵 및 라벨 매칭 확인...
클래스 개수: 51
첫 번째 클래스: bow (인덱스 0)
마지막 클래스: writing (인덱스 50)

첫 번째 비디오: 0002-L
어노테이션 개수: 20
  어노테이션 0: salute - [182, 235]
    클래스맵 인덱스: 31
  어노테이션 1: check time (from watch) - [454, 510]
    클래스맵 인덱스: 3
  어노테이션 2: rub two hands together - [746, 840]
    클래스맵 인덱스: 30


In [20]:
# ─────────────────────────────────────────────────────────────
# 프레임·라벨·마스크 매칭 검증 스크립트 (수정 완료 버전)
# ─────────────────────────────────────────────────────────────

print("\n프레임-라벨 매칭 검증...")

# ─────────── 1. GT 세그먼트 & 라벨 ───────────
if {"gt_segments", "gt_labels"} <= first_sample.keys():
    gt_segments = first_sample["gt_segments"]
    gt_labels   = first_sample["gt_labels"]

    print(f"GT 세그먼트 개수 : {len(gt_segments)}")
    print(f"GT 라벨   개수   : {len(gt_labels)}")

    if len(gt_segments) == len(gt_labels):
        print("✓ 세그먼트와 라벨 개수가 일치합니다.")
        for i, (seg, lab) in enumerate(zip(gt_segments, gt_labels)):
            s, e = seg
            lab_name = class_names[lab] if 0 <= lab < len(class_names) else "OUT_OF_RANGE"
            print(f"  #{i:02d}  프레임 {s:.1f} ~ {e:.1f}  |  라벨 {lab} → {lab_name}")
    else:
        print("✗ 세그먼트와 라벨 개수가 일치하지 않습니다!")
else:
    print("GT 정보가 없습니다.")

# ─────────── 2. 입력 & 마스크 ───────────
if {"inputs", "masks"} <= first_sample.keys():
    inputs = first_sample["inputs"]   # (N,C,T,H,W) or (C,T,H,W)
    masks  = first_sample["masks"]    # (T,)  bool / int

    print(f"\n입력 텐서 형태 : {inputs.shape}")
    print(f"마스크 텐서 형태 : {masks.shape}")

    # ---- 프레임(T) 차원 찾기 ----
    if inputs.dim() == 5:            # (N,C,T,H,W)
        num_frames = inputs.shape[2]
    elif inputs.dim() == 4:          # (C,T,H,W)
        num_frames = inputs.shape[1]
    else:
        raise ValueError(f"예상치 못한 입력 shape: {inputs.shape}")

    print(f"입력 프레임 수 : {num_frames}")

    # ---- 마스크 유효 프레임 수 ----
    valid_frames = masks.float().sum().item()
    print(f"유효한 프레임 수 : {int(valid_frames)}")

    if int(valid_frames) == num_frames:
        print("✓ 마스크가 입력 프레임 수와 일치합니다.")
    else:
        print(f"✗ 불일치! inputs={num_frames}, masks={int(valid_frames)}")
else:
    print("inputs 또는 masks 키가 없습니다.")



프레임-라벨 매칭 검증...
GT 세그먼트 개수 : 2
GT 라벨   개수   : 2
✓ 세그먼트와 라벨 개수가 일치합니다.
  #00  프레임 51.0 ~ 107.0  |  라벨 3 → check time (from watch)
  #01  프레임 343.0 ~ 437.0  |  라벨 30 → rub two hands together

입력 텐서 형태 : torch.Size([1, 3, 512, 160, 160])
마스크 텐서 형태 : torch.Size([512])
입력 프레임 수 : 512
유효한 프레임 수 : 512
✓ 마스크가 입력 프레임 수와 일치합니다.


In [28]:
# 여러 샘플에서 일관성 확인
print("\n여러 샘플 일관성 확인...")
num_test_samples = min(5, len(train_dataset))

for i in range(num_test_samples):
    sample = train_dataset[i]
    print(f"\n샘플 {i}:")
    
    if 'inputs' in sample:
        inputs = sample['inputs']
        print(f"  입력 형태: {inputs.shape}")
    
    if 'gt_segments' in sample and 'gt_labels' in sample:
        segments = sample['gt_segments']
        labels = sample['gt_labels']
        print(f"  세그먼트 개수: {len(segments)}, 라벨 개수: {len(labels)}")
        
        # 라벨 범위 확인
        if len(labels) > 0:
            min_label = labels.min().item()
            max_label = labels.max().item()
            print(f"  라벨 범위: {min_label} ~ {max_label}")
            
            if min_label < 0 or max_label >= len(class_names):
                print(f"  ⚠️ 라벨 범위 문제!")
            else:
                print(f"  ✓ 라벨 범위 정상")

print(f"\n총 {num_test_samples}개 샘플 테스트 완료")


여러 샘플 일관성 확인...

샘플 0:
  입력 형태: torch.Size([1, 3, 512, 160, 160])
  세그먼트 개수: 2, 라벨 개수: 2
  라벨 범위: 18 ~ 21
  ✓ 라벨 범위 정상

샘플 1:
  입력 형태: torch.Size([1, 3, 512, 160, 160])
  세그먼트 개수: 3, 라벨 개수: 3
  라벨 범위: 3 ~ 37
  ✓ 라벨 범위 정상

샘플 2:
  입력 형태: torch.Size([1, 3, 512, 160, 160])
  세그먼트 개수: 3, 라벨 개수: 3
  라벨 범위: 3 ~ 37
  ✓ 라벨 범위 정상

샘플 3:
  입력 형태: torch.Size([1, 3, 512, 160, 160])
  세그먼트 개수: 1, 라벨 개수: 1
  라벨 범위: 0 ~ 0
  ✓ 라벨 범위 정상

샘플 4:
  입력 형태: torch.Size([1, 3, 512, 160, 160])
  세그먼트 개수: 2, 라벨 개수: 2
  라벨 범위: 36 ~ 47
  ✓ 라벨 범위 정상

총 5개 샘플 테스트 완료


In [23]:
# ─────────────────────────────────────────────────────────────
# DataLoader 반복 테스트 (train / val 공용)
# 리스트·텐서 모두 대응 & 중복 코드 제거
# ─────────────────────────────────────────────────────────────
import torch

def inspect_loader(loader, name="loader", max_batches=3):
    print(f"\n{name} 테스트:")
    for b_idx, batch in enumerate(loader):
        if b_idx >= max_batches:
            break

        print(f"  ▸ 배치 {b_idx}")
        # ---------- inputs ----------
        if "inputs" in batch:
            inp = batch["inputs"]
            print(f"    inputs shape : {tuple(inp.shape)}  dtype={inp.dtype}")

        # ---------- gt_labels ----------
        if "gt_labels" in batch:
            lbls = batch["gt_labels"]
            # 리스트일 수도, 텐서일 수도 있음
            if isinstance(lbls, list):
                lbl_tensor = torch.cat(lbls)
            else:
                lbl_tensor = lbls.view(-1)

            print(f"    labels tensor shape : {tuple(lbl_tensor.shape)}")
            print(f"    label id min~max     : {lbl_tensor.min().item()} ~ {lbl_tensor.max().item()}")

        # ---------- optional keys ----------
        if "gt_segments" in batch:
            segs = batch["gt_segments"]
            n_seg = sum(len(s) for s in segs) if isinstance(segs, list) else segs.numel()
            print(f"    total gt segments    : {n_seg}")

# 실행
inspect_loader(train_loader, "훈련 DataLoader", max_batches=3)
inspect_loader(val_loader,   "검증 DataLoader", max_batches=3)

print("\n데이터로더 반복 테스트 완료 ✅")



훈련 DataLoader 테스트:
  ▸ 배치 0
    inputs shape : (2, 1, 3, 512, 160, 160)  dtype=torch.uint8
    labels tensor shape : (3,)
    label id min~max     : 15 ~ 41
    total gt segments    : 3
  ▸ 배치 1
    inputs shape : (2, 1, 3, 512, 160, 160)  dtype=torch.uint8
    labels tensor shape : (4,)
    label id min~max     : 15 ~ 45
    total gt segments    : 4
  ▸ 배치 2
    inputs shape : (2, 1, 3, 512, 160, 160)  dtype=torch.uint8
    labels tensor shape : (4,)
    label id min~max     : 1 ~ 42
    total gt segments    : 4

검증 DataLoader 테스트:
  ▸ 배치 0
    inputs shape : (2, 1, 3, 512, 160, 160)  dtype=torch.uint8
    labels tensor shape : (6,)
    label id min~max     : 17 ~ 26
    total gt segments    : 6
  ▸ 배치 1
    inputs shape : (2, 1, 3, 512, 160, 160)  dtype=torch.uint8
    labels tensor shape : (5,)
    label id min~max     : 11 ~ 23
    total gt segments    : 5
  ▸ 배치 2
    inputs shape : (2, 1, 3, 512, 160, 160)  dtype=torch.uint8
    labels tensor shape : (6,)
    label id min~max   

In [24]:
# 전체 테스트 결과 요약
print("\n" + "="*60)
print("데이터셋 및 데이터로더 Unit Test 결과 요약")
print("="*60)

print(f"✓ 훈련 데이터셋: {len(train_dataset)} 개 샘플")
print(f"✓ 검증 데이터셋: {len(val_dataset)} 개 샘플") 
print(f"✓ 테스트 데이터셋: {len(test_dataset)} 개 샘플")

print(f"✓ 클래스 개수: {len(class_names)}")
print(f"✓ 설정 파일: {config_path}")

# 메모리 사용량 확인
if torch.cuda.is_available():
    print(f"✓ GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"✓ GPU 메모리 사용량: {torch.cuda.memory_allocated() / 1024**3:.1f} GB")

print("\n모든 테스트가 성공적으로 완료되었습니다!")
print("데이터셋과 데이터로더가 올바르게 작동하고 있습니다.")


데이터셋 및 데이터로더 Unit Test 결과 요약
✓ 훈련 데이터셋: 831 개 샘플
✓ 검증 데이터셋: 1958 개 샘플
✓ 테스트 데이터셋: 2370 개 샘플
✓ 클래스 개수: 51
✓ 설정 파일: configs/adatad/pku_mmd/e2e_pku_mmd_videomae_s_768x1_160_adapter copy.py
✓ GPU 메모리: 24.0 GB
✓ GPU 메모리 사용량: 0.0 GB

모든 테스트가 성공적으로 완료되었습니다!
데이터셋과 데이터로더가 올바르게 작동하고 있습니다.
